In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [1]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

2023-05-23 16:31:03.899750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-23 16:31:03.899769: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Goal: Make ws10 feature

### 0. Basics

In [3]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [39]:
# Variables to make testing easier --> determines variable and lead_time
var = 0  # 0,...,4
lead_time = 1  # 0,...,30
forecast_date = 0
lat = 0
lon = 0

<IPython.core.display.Javascript object>

### 1. Load Dataset

In [5]:
dat_raw = ldr.load_data_raw() # list length 5 with 2018 - 2022

# processed data
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

### 2. Test if mean is enough, or have to do everything from scratch?

In [44]:
u10_test_ens = (
    dat_raw[0]
    .isel(forecast_date=forecast_date, lead_time=lead_time, var=0, lat=lat, lon=lon)
    .values
)
v10_test_ens = (
    dat_raw[0]
    .isel(forecast_date=forecast_date, lead_time=lead_time, var=1, lat=lat, lon=lon)
    .values
)
u10_test_mean = (
    dat_train_proc_norm[0]
    .u10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=0
    )
    .values
)
v10_test_mean = (
    dat_train_proc_norm[1]
    .v10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=0
    )
    .values
)
u10_test_std = (
    dat_train_proc_norm[0]
    .u10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=1
    )
    .values
)
v10_test_std = (
    dat_train_proc_norm[1]
    .v10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=1
    )
    .values
)

<IPython.core.display.Javascript object>

### 3. From raw data make ws10

In [40]:
u10_X_ens = (
    dat_raw[0]
    .predictions.isel(
        forecast_date=forecast_date, lead_time=lead_time, var=0, lat=lat, lon=lon
    )
    .values
)
v10_X_ens = (
    dat_raw[0]
    .predictions.isel(
        forecast_date=forecast_date, lead_time=lead_time, var=1, lat=lat, lon=lon
    )
    .values
)

<IPython.core.display.Javascript object>

In [46]:
u10_X_ens[0]

-0.1668648

<IPython.core.display.Javascript object>

In [29]:
dat_raw[0]

<xarray.Dataset>
Dimensions:       (forecast_date: 357, lead_time: 32, var: 5, lat: 120,
                   lon: 130, ens: 50)
Dimensions without coordinates: forecast_date, lead_time, var, lat, lon, ens
Data variables:
    ground_truth  (forecast_date, lead_time, var, lat, lon) float32 ...
    predictions   (forecast_date, ens, lead_time, var, lat, lon) float32 ...

<IPython.core.display.Javascript object>